In [1]:
data = {
    "Producer": "03444266d64f401ef82b234e361cd59419fcc1bf841942",
    "Author": "3e454079995747bb2d7f6b",
    "CreationDate": "0a1713248b091f41be223229744488d8129899b8e46d23",
    "ModDate": "0a1713248b091f41be223229744488d8129899b8e46d23",
    "Title": "2942423ac95849",
    "Hint": "18444671d7d45d1d",
    "Secret": "b0d22125b97b2f258c570361446fbbe73984aeafd415048cfbe59cfaf6dce4ff7e0753b342b1ec1622682200031f045ad04128348373f50933dc4a455c6a69d60f6dc285235fee5a0a5de83ae27b8c9680b595411ca262fc06d9942da42125c3e4fd850bc2b30ac3b68841e6adb8d9f6c2ea"
}

# Chuyển sang bytes
data_bytes = {k: bytes.fromhex(v) for k, v in data.items()}

# In ra kiểm tra
for k, v in data_bytes.items():
    print(f"{k}: {v} (len={len(v)})")

Producer: b'\x03DBf\xd6O@\x1e\xf8+#N6\x1c\xd5\x94\x19\xfc\xc1\xbf\x84\x19B' (len=23)
Author: b'>E@y\x99WG\xbb-\x7fk' (len=11)
CreationDate: b'\n\x17\x13$\x8b\t\x1fA\xbe"2)tD\x88\xd8\x12\x98\x99\xb8\xe4m#' (len=23)
ModDate: b'\n\x17\x13$\x8b\t\x1fA\xbe"2)tD\x88\xd8\x12\x98\x99\xb8\xe4m#' (len=23)
Title: b')BB:\xc9XI' (len=7)
Hint: b'\x18DFq\xd7\xd4]\x1d' (len=8)
Secret: b'\xb0\xd2!%\xb9{/%\x8cW\x03aDo\xbb\xe79\x84\xae\xaf\xd4\x15\x04\x8c\xfb\xe5\x9c\xfa\xf6\xdc\xe4\xff~\x07S\xb3B\xb1\xec\x16"h"\x00\x03\x1f\x04Z\xd0A(4\x83s\xf5\t3\xdcJE\\ji\xd6\x0fm\xc2\x85#_\xeeZ\n]\xe8:\xe2{\x8c\x96\x80\xb5\x95A\x1c\xa2b\xfc\x06\xd9\x94-\xa4!%\xc3\xe4\xfd\x85\x0b\xc2\xb3\n\xc3\xb6\x88A\xe6\xad\xb8\xd9\xf6\xc2\xea' (len=114)


In [2]:
import re
import zlib
from hashlib import md5
from binascii import unhexlify

# --- Cấu hình ---
pdf_file = "share5.pdf"
output_pdf = "share5_decrypted.pdf"
password = b"ctf_2025"

# --- Padding password PDF ---
def pad_password(pwd: bytes) -> bytes:
    padding = bytes([
        0x28,0xBF,0x4E,0x5E,0x4E,0x75,0x8A,0x41,0x64,0x00,0x4E,0x56,0xFF,0xFA,0x01,0x08,
        0x2E,0x2E,0x00,0xB6,0xD0,0x68,0x3E,0x80,0x2F,0x0C,0xA9,0xFE,0x64,0x53,0x69,0x7A
    ])
    pwd = pwd[:32] + padding[:32-len(pwd)]
    return pwd

# --- RC4 key derivation (PDF V2, R3) ---
def compute_rc4_key(user_pwd: bytes, O_hex: str, P: int) -> bytes:
    padded = pad_password(user_pwd)
    O_bytes = unhexlify(O_hex)
    P_bytes = P.to_bytes(4, byteorder='little', signed=True)
    data = padded + O_bytes + P_bytes
    key = md5(data).digest()
    for _ in range(50):
        key = md5(key).digest()
    return key[:16]

# --- RC4 ---
def rc4(key: bytes, data: bytes) -> bytes:
    S = list(range(256))
    j = 0
    out = bytearray()
    # KSA
    for i in range(256):
        j = (j + S[i] + key[i % len(key)]) % 256
        S[i], S[j] = S[j], S[i]
    # PRGA
    i = j = 0
    for byte in data:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        out.append(byte ^ S[(S[i] + S[j]) % 256])
    return bytes(out)

# --- Đọc PDF ---
with open(pdf_file, "rb") as f:
    pdf_data = f.read()

# --- Lấy metadata quan trọng ---
metadata_match = re.search(rb'2 0 obj\s*(<<.*?>>)', pdf_data, re.DOTALL)
metadata = {}
if metadata_match:
    block = metadata_match.group(1)
    for field in [b"Producer", b"Author", b"Title", b"Hint", b"Secret"]:
        m = re.search(rb'/%s\s*<([0-9a-fA-F]+)>' % field, block)
        if m:
            metadata[field.decode()] = m.group(1).decode()

print("Metadata quan trọng:")
for k,v in metadata.items():
    print(f"{k}: {v}")

# --- Lấy thông tin Encrypt (object 54) ---
encrypt_match = re.search(rb'54 0 obj\s*<<.*?/O <([0-9a-fA-F]+)>.*?/P (\-?\d+).*?>>', pdf_data, re.DOTALL)
if not encrypt_match:
    raise ValueError("Không tìm thấy đối tượng Encrypt!")
O_hex = encrypt_match.group(1).decode()
P = int(encrypt_match.group(2))
rc4_key = compute_rc4_key(password, O_hex, P)
print(f"RC4 key derived: {rc4_key.hex()}")

# --- Giải mã các stream ---
stream_pattern = re.compile(rb'(\d+) (\d+) obj\s*<<.*?/Filter /FlateDecode.*?/Length (\d+).*?>>\s*stream\s*(.*?)\s*endstream', re.DOTALL)
output_pdf_bytes = bytearray(pdf_data)  # tạo bản sao để chỉnh sửa

for obj_num, gen_num, length_str, stream_data in re.findall(stream_pattern, pdf_data):
    obj_num_i = int(obj_num)
    gen_num_i = int(gen_num)
    length = int(length_str)
    stream_bytes = stream_data[:length]
    # RC4 object-specific key: RC4(key + obj_num + gen_num)
    obj_key = md5(rc4_key + obj_num_i.to_bytes(3,'little') + gen_num_i.to_bytes(2,'little')).digest()[:16]
    decrypted = rc4(obj_key, stream_bytes)
    try:
        decompressed = zlib.decompress(decrypted)
        final_data = decompressed
    except Exception:
        final_data = decrypted
    # Thay thế stream gốc bằng dữ liệu giải mã (chỉ demo, vẫn giữ PDF structure)
    # Ở đây chỉ in ra, bạn có thể ghi trực tiếp nếu muốn
    print(f"Stream object {obj_num_i} ({length} bytes) giải mã xong, {len(final_data)} bytes")

# --- Ghi file PDF kết quả (demo, vẫn giữ cấu trúc cũ) ---
with open(output_pdf, "wb") as f:
    f.write(pdf_data)  # nếu muốn ghi file giải mã đầy đủ, cần rebuild PDF với stream mới

print(f"File PDF chưa đầy đủ RC4+FlateDecode lưu tại: {output_pdf}")


Metadata quan trọng:
Producer: 03444266d64f401ef82b234e361cd59419fcc1bf841942
Author: 3e454079995747bb2d7f6b
Title: 2942423ac95849
Hint: 18444671d7d45d1d
Secret: b0d22125b97b2f258c570361446fbbe73984aeafd415048cfbe59cfaf6dce4ff7e0753b342b1ec1622682200031f045ad04128348373f50933dc4a455c6a69d60f6dc285235fee5a0a5de83ae27b8c9680b595411ca262fc06d9942da42125c3e4fd850bc2b30ac3b68841e6adb8d9f6c2ea
RC4 key derived: 48624ebf119f41e0b6f1316b1db66287
Stream object 1 (30071 bytes) giải mã xong, 30071 bytes
Stream object 6 (21873 bytes) giải mã xong, 21873 bytes
Stream object 7 (22113 bytes) giải mã xong, 22113 bytes
Stream object 8 (13397 bytes) giải mã xong, 13397 bytes
Stream object 9 (24722 bytes) giải mã xong, 24722 bytes
Stream object 12 (24321 bytes) giải mã xong, 24321 bytes
Stream object 13 (24997 bytes) giải mã xong, 24997 bytes
Stream object 14 (23421 bytes) giải mã xong, 23421 bytes
Stream object 15 (24106 bytes) giải mã xong, 24106 bytes
Stream object 16 (24149 bytes) giải mã xong, 24149 

In [5]:
import PyPDF2
import zlib

pdf_path = "share5.pdf"
password = "ctf_2025"

with open(pdf_path, "rb") as f:
    reader = PyPDF2.PdfReader(f)

    # Giải mã PDF
    if reader.is_encrypted:
        if reader.decrypt(password) == 0:
            raise ValueError("Password không đúng!")

    print(f"Số trang PDF: {len(reader.pages)}")

    for i, page in enumerate(reader.pages, 1):
        print(f"\n--- Trang {i} ---")
        try:
            text = page.extract_text()
            if text:
                print(text)
            else:
                contents = page["/Contents"]

                # Nếu là list các object, resolve từng object
                if isinstance(contents, list):
                    for obj in contents:
                        stream = obj.get_object()  # <-- resolve IndirectObject
                        data = stream.get_data()
                        try:
                            text_decoded = zlib.decompress(data)
                            print(text_decoded.decode(errors="ignore"))
                        except:
                            print("Không giải nén được stream")
                else:
                    stream = contents.get_object()
                    data = stream.get_data()
                    try:
                        text_decoded = zlib.decompress(data)
                        print(text_decoded.decode(errors="ignore"))
                    except:
                        print("Không giải nén được stream")
        except Exception as e:
            print("Lỗi xử lý trang:", e)


Số trang PDF: 5

--- Trang 1 ---
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream

--- Trang 2 ---
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream

--- Trang 3 ---
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream

--- Trang 4 ---
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream
Không giải nén được stream

--- Trang 5 ---
Không giải nén được stream
Không giải nén được stream
Không gi